# <u>**Streaming Service Comparison**</u>

### **Objective:**
- Determine which streaming platform hosts the majority of content I enjoy so that I can pare down the services to which I subscribe. 

### **Data Sources:**
- [Netflix via Kaggle.com](https://www.kaggle.com/datasets/octopusteam/full-netflix-dataset)
- [Hulu via Kaggle.com](https://www.kaggle.com/datasets/octopusteam/full-hulu-dataset)
- [Prime via Kaggle.com](https://www.kaggle.com/datasets/octopusteam/full-amazon-prime-dataset/data)
- [AppleTV via Kaggle.com](https://www.kaggle.com/datasets/octopusteam/full-apple-tv-dataset)
- [Streaming Service Pricing](https://github.com/rkynhoff/Streaming_Service_Comparisons/blob/main/Data/Streaming_Services_Pricing_Histories.csv)
    - Created manually using  

## **Data Collection & Loading**

### **Import Pandas, Numpy, Matplotlib, Wordcloud, and PIL**

In [135]:
import glob
import pandas as pd
import os
import numpy as np
import pyspark
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sn
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from PIL import Image
import tabulate

### **Data Load**

In [136]:
# Load in each file separately. 
apple = pd.read_csv("Data/AppleTV.csv")
hulu = pd.read_csv("Data/Hulu.csv")
netflix = pd.read_csv("Data/Netflix.csv")
prime = pd.read_csv("Data/Prime.csv")
prices = pd.read_csv("Data/Streaming_Services_Pricing_Histories.csv")


### **Data Dictionary**
This is the data dictionary associated with the four original streaming service DataFrames. 				

| Field/Column  | Description | Source | Data Type | Example |
|:--------------|:------------|:-------|:----------|:--------|
| title | The name of the movie or tv show. | Original csv files  | text string | Breakfast at Tiffany's |
| type | The title content type, either "move" or "tv" | Original csv files | text string | movie |
| genres | Includes all available genres for the title. | Original csv files | text string  | Comedy, Drama, Romance |
| releaseYear | The year the movie or tv show was released according to the streaming platform. | Original csv files | int | 1961 |
| imdbId | The IMDb ID, if present, a unique value used in the URL of the IMDb content listing. | Original csv files | string | tt0054698 |
| imdbAverageRating | The average rating by customers on IMDb | Original csv files | float | 7.6 |
| imdbNumVotes | The total number of votes by customers on IMDb | Original csv files | int | 197447 |
| availableCountries | List of countries in which the title is available. | Original csv files | text string | CA, US |

In [137]:
prices.dtypes

Service_Name     object
Date             object
Price           float64
dtype: object

### **Preliminary Data Exploration**

In [138]:
display(prices)

,Service_Name,Date,Price
0,AppleTV,Nov-19,4.99
1,AppleTV,Dec-19,4.99
2,AppleTV,Jan-20,4.99
3,AppleTV,Feb-20,4.99
4,AppleTV,Mar-20,4.99
...,...,...,...
251,Prime,Oct-24,14.99
252,Prime,Nov-24,14.99
253,Prime,Dec-24,14.99
254,Prime,Jan-25,14.99


In [139]:
prices.dtypes

Service_Name     object
Date             object
Price           float64
dtype: object

The prices df appears to have a date column in month/year. I will need to reformat that date field.

In [140]:
# View the head of each service dataframe 
# Create a variable to include a list of all dataframes recently imported
services = [apple, hulu, netflix, prime]
# Display only the head of each dataframe separately
for s in services:
    display(s.head())

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries
0,Four Rooms,movie,Comedy,1995.0,tt0113101,6.7,113546.0,NaN
1,Forrest Gump,movie,"Drama, Romance",1994.0,tt0109830,8.8,2354158.0,NaN
2,American Beauty,movie,Drama,1999.0,tt0169547,8.3,1241156.0,NaN
3,Citizen Kane,movie,"Drama, Mystery",1941.0,tt0033467,8.3,478085.0,NaN
4,Metropolis,movie,"Drama, Sci-Fi",1927.0,tt0017136,8.3,192628.0,NaN


,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries
0,Ariel,movie,"Comedy, Crime, Romance",1988.0,tt0094675,7.4,8991.0,NaN
1,Shadows in Paradise,movie,"Comedy, Drama, Music",1986.0,tt0092149,7.5,7792.0,NaN
2,Finding Nemo,movie,"Adventure, Animation, Comedy",2003.0,tt0266543,8.2,1149529.0,NaN
3,Forrest Gump,movie,"Drama, Romance",1994.0,tt0109830,8.8,2354158.0,NaN
4,The Fifth Element,movie,"Action, Adventure, Sci-Fi",1997.0,tt0119116,7.6,522699.0,NaN


,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries
0,American Beauty,movie,Drama,1999.0,tt0169547,8.3,1241156.0,NaN
1,The Fifth Element,movie,"Action, Adventure, Sci-Fi",1997.0,tt0119116,7.6,522699.0,NaN
2,Kill Bill: Vol. 1,movie,"Action, Crime, Thriller",2003.0,tt0266697,8.2,1238778.0,NaN
3,Jarhead,movie,"Biography, Drama, War",2005.0,tt0418763,7.0,214024.0,NaN
4,Unforgiven,movie,"Drama, Western",1992.0,tt0105695,8.2,449594.0,NaN


,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries
0,Ariel,movie,"Comedy, Crime, Romance",1988.0,tt0094675,7.4,8991.0,NaN
1,Four Rooms,movie,Comedy,1995.0,tt0113101,6.7,113546.0,NaN
2,Judgment Night,movie,"Action, Crime, Drama",1993.0,tt0107286,6.6,19686.0,NaN
3,Forrest Gump,movie,"Drama, Romance",1994.0,tt0109830,8.8,2354158.0,NaN
4,Citizen Kane,movie,"Drama, Mystery",1941.0,tt0033467,8.3,478085.0,NaN


In [147]:
apple.value_counts("availableCountries")

availableCountries
US                                40
CA                                13
CA, US                             7
GB                                 7
ES                                 3
AU, CA, NO, US                     2
GB, US                             2
DE                                 2
AR, BR, CL, CO, MX                 1
AT, CH, DE, FR, GB, IE, IT, US     1
AU, US                             1
AU                                 1
FR                                 1
IN                                 1
Name: count, dtype: int64

In [141]:
# Get a sum of the total row count from each dataframe
total_rows = len(apple) + len(hulu) + len(netflix) + len(prime)
print(f"The total row count is {total_rows} for all dataframes.")

The total row count is 119711 for all dataframes.


### **Initial Insights**
- There are a total of 119,711 among all raw dataframes
- Each dataframe contains the same column headings
- None of the dataframes contain a column for the source of the dataframe
- All numerical columns appear to be floats and don't need to be
    - Change ***releaseYear*** and ***imdbNumVotes*** to integers
- The majority of the titles displayed contain multiple genres in the ***genres*** column
- The ***imdbId*** columns appear to match 
    - *American Beauty* in the 1st and 3rd dataframe
    - *Forest Gump* in the 1st, 2nd and 4th dataframes
- Of all the rows displayed, none include ***availableCountries*** data
    - Further information proves that this data point is not prevelent throughout the four dataframes
- The prices df appears to have a date column in month/year.
    - I will need to reformat that date field.


### **Preliminary Data Manipulation**

In [142]:
# Add a column to each dataframe to identify the data source since individually they do not have an identifier. 
# This is to ensure the data is discernable after the merge when duplicate content is present
apple["Service_Name"] = "AppleTV"
apple["Price"] = 9.99
hulu["Service_Name"] = "Hulu"
hulu["Price"] = 9.99
netflix["Service_Name"] = "Netflix"
netflix["Price"] = 6.99
prime["Service_Name"] = "Prime"
prime["Price"] = 14.99


# Using a previously made variable, created a for loop to run through each dataframe and display only the first two rows of each
# This is to ensure the column names and values were added propertly
for s in services:
    display(s.head(2))

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries,Service_Name,Price
0,Four Rooms,movie,Comedy,1995.0,tt0113101,6.7,113546.0,NaN,AppleTV,9.99
1,Forrest Gump,movie,"Drama, Romance",1994.0,tt0109830,8.8,2354158.0,NaN,AppleTV,9.99


,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries,Service_Name,Price
0,Ariel,movie,"Comedy, Crime, Romance",1988.0,tt0094675,7.4,8991.0,NaN,Hulu,9.99
1,Shadows in Paradise,movie,"Comedy, Drama, Music",1986.0,tt0092149,7.5,7792.0,NaN,Hulu,9.99


,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries,Service_Name,Price
0,American Beauty,movie,Drama,1999.0,tt0169547,8.3,1241156.0,NaN,Netflix,6.99
1,The Fifth Element,movie,"Action, Adventure, Sci-Fi",1997.0,tt0119116,7.6,522699.0,NaN,Netflix,6.99


,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries,Service_Name,Price
0,Ariel,movie,"Comedy, Crime, Romance",1988.0,tt0094675,7.4,8991.0,NaN,Prime,14.99
1,Four Rooms,movie,Comedy,1995.0,tt0113101,6.7,113546.0,NaN,Prime,14.99


### **Data Merge**

In [143]:
# Merge the dataframes together into one by appending them to the end of each other since they are all using the same column names
streaming_v1 = pd.concat([apple, hulu, netflix, prime], axis = 0)

# Based on above initial review, check to ensure there are a total of 119,711 rows in the new dataframe
streaming_v1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 119711 entries, 0 to 70293
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   title               116151 non-null  object 
 1   type                119711 non-null  object 
 2   genres              116226 non-null  object 
 3   releaseYear         119419 non-null  float64
 4   imdbId              109192 non-null  object 
 5   imdbAverageRating   105942 non-null  float64
 6   imdbNumVotes        105942 non-null  float64
 7   availableCountries  651 non-null     object 
 8   Service_Name        119711 non-null  object 
 9   Price               119711 non-null  float64
dtypes: float64(4), object(6)
memory usage: 10.0+ MB


### **Data Cleaning & Preparation**

In [ ]:
# Drop the availableCountries column given that less than 1% (0.0054%) of entries utilize this data element
streaming_v1.drop("availableCountries", axis = 1, inplace=True)

# Display only the columns to ensure the dropped column is gone
streaming_v1.columns

In [ ]:
# Rename the columns 
streaming_v1.rename(columns={"title":"Title","type":"Content_Type","genres":"Combined_Genres","releaseYear":"Release_Year","imdbId":"IMDb_ID","imdbAverageRating":"IMDb_Avg_Rating","imdbNumVotes":"IMDb_Votes_Count"},inplace = True)
streaming_v1.columns


In [ ]:
# Change Release_Year, IMDb_Votes_Counts from floats to integers 
streaming_v1[["Release_Year","IMDb_Votes_Count"]] = streaming_v1[["Release_Year","IMDb_Votes_Count"]].apply(np.int64)
# Change Service_Name & Combined_Genres to string
streaming_v1[["Service_Name","Combined_Genres"]] == streaming_v1[["Service_Name","Combined_Genres"]].astype(str)

# Display only the data types to ensure the data types were changed
streaming_v1.info()

In [ ]:
# Determine how many titles contain null values
print("There are",streaming_v1["Title"].isna().sum(),"rows with null values in the Title column.")

In [ ]:
# Drop the rows containing null values in the Title column
streaming_v1 = streaming_v1.dropna(subset=["Title"])

# Ensure duplicate rows were dropped
streaming_v1.info()

# Can see in the results that the total entries went from 119,711 to 116,151, a drop of 3,560 rows


In [ ]:
# Determine how many duplicated rows exist
print("There are", streaming_v1.duplicated().sum(),"duplicated rows.")

In [ ]:
# Taking a look at the duplicated rows since there are so few & sort by title alphabetically with the default ascending
streaming_v1[streaming_v1.duplicated(keep=False)].sort_values("Title")

In [ ]:
# Since the rows do appear to be fully dupliated, I will drop the duplicate rows here
streaming_v2 = streaming_v1.drop_duplicates()

# Reset the index
streaming_v2 = streaming_v2.reset_index(drop=True)

# View total of entires, which should equal 116143 (116151  - 8)
streaming_v2.info()

In [ ]:
# Determine how man rows include null values for Combined_Genres
print("There are",streaming_v1["Combined_Genres"].isna().sum(),"rows with null values in the Combined_Genres column.")

In [ ]:
# View the rows with null values for Combined_Genres
# May need to address this later
streaming_v1.loc[streaming_v1["Combined_Genres"].isnull()]

In [ ]:
# Now that duplicated rows are gone, I want to see if there might be duplicated content baesd on IMDb_ID, Release_Year, Title, Content_Type, and Service Name
streaming_v2[streaming_v2.duplicated(subset = ["IMDb_ID","Title","Release_Year","Content_Type","Service_Name"], keep = False)].sort_values(by=["Title","Service_Name","Combined_Genres"], ascending=[True,True,True])

In [ ]:
# Based on the output above, I can see there is duplicated content and rows where genres could be combined into one row for each Service_Name

# Remove the duplicated rows that have a null value in the Combined_Genres column by index number
streaming_v3 = streaming_v2.drop([41702,12177,37721,89252,108129,108076,40887,102086,107404])
# Reset index
streaming_v3 = streaming_v3.reset_index(drop=True)
# Double check
streaming_v3[streaming_v3["Title"] == "The Final Ride"]


In [ ]:
# Redisplay the duplicated content baesd on IMDb_ID, Release_Year, Title, Content_Type, and Service Name

grouped_dupes = streaming_v3[streaming_v3.duplicated(subset = ["IMDb_ID","Title","Release_Year","Content_Type","Service_Name"], keep = False)].sort_values(by=["Title","Service_Name","Combined_Genres"], ascending=[True,True,True])
grouped_dupes

In [ ]:
# See AI_Notes.docx for more information on how this code block came about
# Initialize an empty dictionary in which to store concatenated genres
concatenated_genres = {}

# Iterate over each row in the grouped_dupes dataframe
for index, row in grouped_dupes.iterrows():
    # Createa a unique key for each row in the grouped_dupes dataframe by combining certain columns
    key = (row["Title"], row["Content_Type"],row["Release_Year"], row["IMDb_ID"], row["IMDb_Votes_Count"], row["Service_Name"])
    # Checks if the key exists in the concatenated_genres dictionary & if so, append the new genres from the currnt row to the existing list of genres in the dictionary with a comma and space after each
    if key in concatenated_genres:
        concatenated_genres[key] += row["Combined_Genres"] + ', '
    # If the key doesn't exist in the dictionary, create a new entry in the dictionary with the combined genres from the current row, with a comma and a space
    else:
        concatenated_genres[key] = row["Combined_Genres"] + ', '

# Iniitialize a list to 
result_data = []
# Iterate over each key-value pair in the concatenated_genres dictionary
for key, genres in concatenated_genres.items():
    # Append a new list to the result_data list which contains the key values and the genres with the trailing comma and space removed
    result_data.append([key[0], key[1], genres[:-2], key[2], key[3], key[4], key[5]])

# Create a new dataframe from the result_data list with specified column names
result_df = pd.DataFrame(result_data, columns = ["Title","Content_Type","Combined_Genres","Release_Year","IMDb_ID","IMDb_Votes_Count","Service_Name"])
# Create a new column in the new dataframe and assign the value of "none"
# This was done because each time I attempted to add the IMDb_Avg_Rating column to the key list above, the code would fail because it is an integer column in the original dataframe
result_df["IMDb_Avg_Rating"] = "None"
# Define a list of columns in the order in which I want them in the new dataframe
column_titles = ["Title","Content_Type","Combined_Genres","Release_Year","IMDb_ID","IMDb_Avg_Rating","IMDb_Votes_Count","Service_Name"]
# reorder the columns in the new dataframe to match the specified order in the above code line
result_df = result_df.reindex(columns=column_titles)
# Display the new dataframe
display(result_df)

In [ ]:
# Append the newly created dataframe to the previous dataframe
streaming_v4 = pd.concat([streaming_v3,result_df], axis = 0)

# Reset the indeces
streaming_v4.reset_index(drop=True) 

In [ ]:
# Check to ensure that one of the titles with a duplicate row matching title, content_type, release_year, and service_name, but with a null value is Genres is present with the correct combined genres
streaming_v4[streaming_v4["Title"] == "The Final Ride"]

In [ ]:
display(streaming_v4[streaming_v4["Title"] == "Aurora"])
display(streaming_v4[streaming_v4["Title"] == "De olhos abertos"]) 
display(streaming_v4[streaming_v4["Title"] == "Fences"])
display(streaming_v4[streaming_v4["Title"] == "Imago"])
display(streaming_v4[streaming_v4["Title"] == "Fences"])
display(streaming_v4[streaming_v4["Title"] == "Leonardo: The Man Who Saved Science"])
display(streaming_v4[streaming_v4["Title"] == "Stealing Chaplin"])
display(streaming_v4[streaming_v4["Title"] == "When life gives you lemons"])

In [ ]:
# streaming_v2[streaming_v2.duplicated(subset = ["IMDb_ID","Title","Release_Year","Content_Type","Service_Name"], keep = False)].sort_values(by=["Title","Service_Name","Combined_Genres"], ascending=[True,True,True])
streaming_v5 = streaming_v4.drop([92152,93061,13327,13556,42318,42623,105038,106205,103994,106324,105665,108035,93553,96736,107192,107203,56996,107581])

In [ ]:
# Checking for the subset duplicates again
streaming_v5[streaming_v5.duplicated(subset = ["Title","Content_Type","Combined_Genres","Release_Year","IMDb_ID","Service_Name"], keep = False)].sort_values(by=["Title","Combined_Genres","Service_Name"], ascending=[True,True,True]).sort_index()

In [ ]:
# Checking a specific title to ensure of dropped rows
display(streaming_v5[streaming_v5["Title"] == "Aurora"])

In [ ]:
streaming_v5.dtypes

In [ ]:
# I see that my alterations to the rows above has changed my data type value, so I need to remove the "None" values from the IMDb_Avg_Rating column
streaming_v5["IMDb_Avg_Rating"] = streaming_v5["IMDb_Avg_Rating"].replace("None",pd.NA)
# streaming_v5.value_counts("IMDb_Avg_Rating")
streaming_v5.dtypes


In [ ]:
# An attempt to remove the duplicated Genre values in the Combined_Genres column prior to splitting the values into separate columns
# def remove_duped_combined_genres(Combined_Genres):
#     genres = Combined_Genres.str.split()
#     unique_comb_genres = list(dict.fromkeys(genres))
#     return ' '.join(unique_comb_genres)

# streaming_v5["Combined_Genres"] = streaming_v5["Combined_Genres"].apply(remove_duped_combined_genres)

In [ ]:
# Second attempt to remove the duplicated Genre values in the Combined_Genres column prior to splitting the values into separate columns
# streaming_v5["Combined_Genres"] = streaming_v5["Combined_Genres"].apply(lambda x: ' '.join(set(x.split())))

In [ ]:
# Still seeing the duplicated word "Comedy" in the COmbined_Genres column
display(streaming_v5[streaming_v5["Title"] == "Breakfast at Tiffany's"])

In [ ]:
streaming_v5.info()

In [ ]:
#Obtain max number of elements in the Genres column
# streaming_v4["Combined_Genres"].str.split(", ",expand=True)


In [ ]:
# Split the elements in the Genres column while creating new columns for the max number 
streaming_v5[["Genre_1","Genre_2","Genre_3","Genre_4","Genre_5","Genre_6","Genre_7"]] = streaming_v5["Combined_Genres"].str.split(',',expand=True)

streaming_v5.head(2)

In [ ]:
# Creating clean dataframe
streaming_clean = streaming_v5

In [ ]:
# Create a final CSV file from the cleaned dataframe to use in Tableau
# Note this is commented out after the first run to create the file so the program doesn't fail

# streaming_clean.to_csv("Streaming_Clean_Final.csv", sep=",", index=False, header=True)

### **Begin Analysis**

### **Data Dictionary**
This is the data dictionary associated with final dataframe, streaming_clean.					

| Field | Description | Source |  Data Type | Example |
|:------|:------------|:-------|:---------- |:--------|
| Title | The name of the movie or tv show. | From the original csv files| text string | Breakfast at Tiffany's |
| Content_Type | The title content, either "movie" or "tv." | Renamed from the original service platform dataframes. | text string | movie |
| Combined_Genres | Includes all available genres in alphabetical order. | Renamed from the original service platform dataframes. | text string | Comedy, Drama, Romance |
| Release_Year | The year the movie or tv show was released according to the streaming platform.  | Renamed from the original service platform dataframes. | int | 1961 |
| IMDb_ID | The IMDb ID, if present, a unique value used in the URL of the IMDb content listing. | Renamed from the original service platform dataframes. | text string | tt0054698 |
| IMDb_Avg_Rating | The average rating by customers on IMDb  | Renamed from the original service platform dataframes. | float | 7.6|
| IMDb_Votes_Count | The total number of votes by customers on IMDb | Renamed from the original service platform dataframes. | int | 197447 |
| Service_Name | The streaming platform name. | Created field based on the service from which the data came. | text string | AppleTV |
| Genre_1 | The first genre in the Combined_Genres field, if present. | Created field from splitting the genres field from the original file into individual columns. | text string | Comedy |
| Genre_2 | The second genre in the Combined_Genres field, if present. | Created field from splitting the genres field from the original file into individual columns. | text string | Drama |
| Genre_3 | The third genre in the Combined_Genres field, if present. | Created field from splitting the genres field from the original file into individual columns. | text string | Romance |
| Genre_4 | The fourth genre in the Combined_Genres field, if present. | Created field from splitting the genres field from the original file into individual columns. | text string | NaN | 
| Genre_5 | The fifth genre in the Combined_Genres field, if present. | Created field from splitting the genres field from the original file into individual columns. | text string | NaN |
| Genre_6 | The sixth genre in the Combined_Genres field, if present. | Created field from splitting the genres field from the original file into individual columns. | text string | NaN |
| Genre_7 | The seventh genre in the Combined_Genres field, if present. | Created field from splitting the genres field from the original file into individual columns. | text string | NaN | 

In [ ]:
streaming_clean.info()

In [ ]:
# Hex Codes for streaming services
# Apple: gray = #A3AAAE, blue = #009DDC
# Hulu: greeen = #1CE783, black = #001908
# Netflix: crimson = #E50914, black = #221F1F
# Prime: orange = #FF9900, blue = #05A0D1, dark blue: #232F3E

In [ ]:
# Crate a bar graph to show distribution of content types from the Content_Type column
# Plot it with orange for "movies" and purple for "tv"
streaming_clean["Content_Type"].value_counts().plot(kind='bar',color=["orange","purple"])
# Add a plot title
plt.title("Streaming Service Content Type Distribution")
# Change the orientation of the x ticks
plt.xticks(rotation = 360)
# Relable the X axis
plt.xlabel("Content Type")
# Show the bar graph 
plt.show()


Content Type Analysis
- Among the four streaming services analyzed, the majority of content is steeped in movies over tv shows
- There appears to be nearly 5 times the amount of movies vs tv shows offered among all four platforms

In [ ]:
# Plotting which streaming platform contains the most content
chart = sn.countplot(streaming_clean, x = "Service_Name", hue = "Service_Name", palette = ["#009DDC","#1CE783","#831010","#FF9900"])
chart.set_ylabel("")
chart.set_xlabel("")
chart.set_title("Total Streaming Content by Platform")
plt.show()


In [ ]:
# streaming_clean["Service_Name"].value_counts().plot(kind='bar',color=["#FF9900","#831010","#A3AAAE","#1CE783"])
# # Add a plot title
# plt.title("Total Streaming Content by Platform")
# # Change the orientation of the x ticks
# plt.xticks(rotation = 360)
# # Show the bar graph 
# plt.show()

In [ ]:
# Counts of the content type by streaming sesrvice

# Hex Codes for streaming services
# Apple: gray = #A3AAAE, blue = #009DDC
# Hulu: greeen = #1CE783, black = #001908
# Netflix: crimson = #D81F26, black = #221F1F
# Prime: orange = #FF9900, blue = #05A0D1, dark blue: #232F3E

# Define the colors for each content_type
# movie_colors = ["#A3AAAE", "#1CE783", "#D81F26", "#A3AAAE"]
# tv_colors = ["#009DDC", "#001908", "#221F1F", "#232F3E"]
sn.countplot(streaming_clean, x = "Service_Name", hue = "Content_Type", palette = ["#A3AAAE", "#1CE783"])
plt.xlabel("Service Name")



In [ ]:
# Let's compare apples to apples and stack the content types to compare total content among the four service providers

# Group by Service_Name and count the number of each Content_Type
cont_type = streaming_clean.groupby("Service_Name")["Content_Type"].value_counts().unstack(fill_value = 0)

# Plot the result of the grouped data above as a stacked bar plot with specific coloring
cont_type.plot(kind =  "bar", stacked = True, color = ["#A3AAAE", "#009DDC"])
# Relable the x axis
plt.xlabel("Service Name")
# Reposition the x labels so they are horizontal with the x axis
plt.xticks(rotation = 360)
# Create a graph title
plt.title("Total Content by Content Type per Provider")
plt.show()

In [ ]:
# Attempt at adding value_count labels 
# Group by Service_Name and count the number of each Content_Type
cont_type = streaming_clean.groupby(["Service_Name", "Content_Type"]).size().reset_index(name="count")

# Plot the result of the grouped data above as a stacked bar plot with specific coloring
cont_typepivot = cont_type.pivot_table(index="Service_Name", columns="Content_Type", values="count", fill_value=0)
cont_typepivot.plot(kind =  "bar", stacked = True, color = ["#A3AAAE", "#009DDC"])

# Relable the x axis
plt.xlabel("Service Name")
# Reposition the x labels so they are horizontal with the x axis
plt.xticks(rotation = 360)
# Create a chart title
plt.title("Total Content by Content Type per Provider")

# Get the height of the bars
bar_heights = cont_typepivot.iloc[0].values

# Get the x positions of the bars
bar_positions = range(len(cont_typepivot.columns))

# Create value count labels inside the bars on the graph
for i, height in enumerate(bar_heights):
    if height > 0:
        plt.text(bar_positions[i], height + 5, str(int(height)), ha='center', va='bottom')

# Show the plot
plt.show()

Content Type Analysis
- At first glance, the Prime content catalog appears to be nearly twice that of other providers
    - This is because much of the content within Prime is either available for rent or through a secondary subscription service

In [ ]:
combined_null = streaming_clean["Combined_Genres"].isnull().sum()

print(f"There are {combined_null} null values in the Combined_Genres column.")

In [ ]:
# Create a value count variable for the top 10 value cgenres in the Combined_Genres column and count them
genre_counts = streaming_clean["Combined_Genres"].value_counts().head(10)
# Plot the genres
genre_counts.plot.bar()

# Creating a loop to put the value count of each genre count into the middle of its respective bar vertically in white font
for p in plt.gca().patches:
    height = p.get_height()
    plt.gca().text(p.get_x() + p.get_width()/2, height/2, str(height), ha = "center", color = "white", rotation = 90)

# create a Combined Genres label on the x axis
plt.xlabel("Combined Genres")
# rotate x-labels to 45 degree angle with a right text orientation
plt.xticks(rotation = 45, ha = "right")
# create a graph title
plt.title("Top 10 Combined Genres Among All Streaming Services")
# Show the plot
plt.show()

Cobmined Genres Analysis
- Of the values in the Combined_Genres column, 2,204 are null. 
- The tope three values within the Combined_Genres column are actually individual genres comprising of # of the overall content among all service providers
- Within the top 5 genres, Drama was paired with Comedy # times and with Romance # times
- While horror is 6th in the list, it is also an individual genre 

In [ ]:
# Looking for the top 10 genres offered among the streaming services based on the first genre listed in the Combined_Genres column parsed into the Genre_1 column
# Create a value count variable for the top 10 value cgenres in the Combined_Genres column and count them
genre_counts = streaming_clean["Genre_1"].value_counts().head(10)
# Plot the genres
genre_counts.plot.bar()

# Creating a loop to put the value count of each genre count into the middle of its respective bar vertically in white font
for i, p in enumerate(plt.gca().patches):
    height = p.get_height()
    # For all bars but last two, show the label inside the bar with white font
    if height < 10: 
        plt.gca().text(p.get_x() + p.get_width()/2, height + 10, str(height), ha = "center", color = "black")
    elif i < len(plt.gca().patches) - 2:
        plt.gca().text(p.get_x() + p.get_width()/2, height/2, str(height), ha = "center", color = "white", rotation = 90)
    else: 
        # For the last two columns, put label above the bar in black font
        plt.gca().text(p.get_x() + p.get_width()/2, height + 10, str(height), ha = "center", color = "black", rotation = 90)

# create a Combined Genres label on the x axis
plt.xlabel("Genres")
# rotate x-labels to 45 degree angle with a right text orientation
plt.xticks(rotation = 45, ha = "right")
# create a graph title
plt.title("Top 10 Individual Genres Among All Streaming Services")
# Show the plot
plt.show()


The top 10 individual genres among all four streaming platforms are 
1. Drama
2. Comedy
3. Action
4. Documentary
5. Crime
6. Horor
7. Adventure
8. Biography
9. Animation
10. Thriller

Not surprisingly, Drama and Comedy make up the top two individual genre categories. Note that the Romance category hasn't made the top 10 of individual genres. 

In [ ]:
# streaming_clean[["Genre_1","IMDb_Avg_Rating"]].value_counts().head(10).plot.bar(x = "Genre_1", y = "IMDb_Avg_Rating")

In [ ]:
# Separate the AppleTV rows from the rest of the df
apple_tv = streaming_clean[streaming_clean["Service_Name"] == "AppleTV"] 
# Group the Genre_1 column and get the total counts for each genre
apple_tv_genres = apple_tv.groupby("Genre_1")["Genre_1"].count().reset_index(name = "Count")
# Ascertain the top 5 genres
top_apple_tv_genres = apple_tv_genres.sort_values("Count", ascending = False).head(6)

plt.figure(figsize = (8, 6))
sn.barplot(x = top_apple_tv_genres["Genre_1"], y = top_apple_tv_genres["Count"])
plt.xlabel("Individual Genres from Genre_1 Column")
plt.ylabel(None)
plt.title("Top 5 AppleTV Genres")

plt.show


In [ ]:
# Group the Genre_1 column and get the total counts for each genre
apple_tv_genres = apple_tv.groupby("Combined_Genres")["Combined_Genres"].count().reset_index(name = "Count")
# Ascertain the top 5 genres
top_apple_tv_genres = apple_tv_genres.sort_values("Count", ascending = False).head(6)

plt.figure(figsize = (8, 6))
sn.barplot(x = top_apple_tv_genres["Combined_Genres"], y = top_apple_tv_genres["Count"])
plt.xlabel("Genres")
plt.ylabel(None)
plt.xticks(rotation = 45)
plt.title("Top 5 AppleTV Combined Genres")

plt.show()

In [ ]:
# Create variable with value counts of each release year
release_yr_cnts = apple_tv["Release_Year"].value_counts()

# Convert the index of the Release Years from integers to strings because WordCloud requires strings inputs
release_yr_cnts.index = release_yr_cnts.index.map(str)

# Create the WordCloud specifying 
apple_tv_wordcloud = WordCloud(width = 800, height = 400, background_color = "black").generate_from_frequencies(release_yr_cnts)

# Setting figure size in inches
plt.figure(figsize = (12, 8))
# Show the created WordCloud with a bilinear interpolation, which is used to estimate the value of pixels used to generate the image
plt.imshow(apple_tv_wordcloud, interpolation = "bilinear")
# Do not display axes
plt.axis("off")
# Create a title for the WordCloud
plt.title("AppleTV Release Years WordCloud")
# Show the plot
plt.show()

In [ ]:
# Separate the Hulu rows from the rest of the df
hulu_tv = streaming_clean[streaming_clean["Service_Name"] == "Hulu"] 
# Group the Genre_1 column and get the total counts for each genre
hulu_tv_genres = hulu_tv.groupby("Genre_1")["Genre_1"].count().reset_index(name = "Count")
# Ascertain the top 5 genres
top_hulu_tv_genres = hulu_tv_genres.sort_values("Count", ascending = False).head(6)

plt.figure(figsize = (8, 6))
sn.barplot(x = top_hulu_tv_genres["Genre_1"], y = top_hulu_tv_genres["Count"])
plt.xlabel("Individual Genres from Genre_1 Column")
plt.ylabel(None)
plt.title("Top 5 Hulu Genres")

plt.show


In [ ]:
# Group the Genre_1 column and get the total counts for each genre
hulu_tv_genres = hulu_tv.groupby("Combined_Genres")["Combined_Genres"].count().reset_index(name = "Count")
# Ascertain the top 5 genres
top_hulu_tv_genres = hulu_tv_genres.sort_values("Count", ascending = False).head(6)

plt.figure(figsize = (8, 6))
sn.barplot(x = top_hulu_tv_genres["Combined_Genres"], y = top_hulu_tv_genres["Count"])
plt.xlabel("Genres")
plt.ylabel(None)
plt.xticks(rotation = 45)
plt.title("Top 5 Hulu Combined Genres")

plt.show()

In [ ]:
# Create variable with value counts of each release year
release_yr_cnts = hulu_tv["Release_Year"].value_counts()

# Convert the index of the Release Years from integers to strings because WordCloud requires strings inputs
release_yr_cnts.index = release_yr_cnts.index.map(str)

# Create the WordCloud specifying 
hulu_tv_wordcloud = WordCloud(width = 800, height = 400, background_color = "black").generate_from_frequencies(release_yr_cnts)

# Setting figure size in inches
plt.figure(figsize = (12, 8))
# Show the created WordCloud with a bilinear interpolation, which is used to estimate the value of pixels used to generate the image
plt.imshow(hulu_tv_wordcloud, interpolation = "bilinear")
# Do not display axes
plt.axis("off")
# Create a title for the WordCloud
plt.title("Hulu Release Years WordCloud")
# Show the plot
plt.show()

In [ ]:
# Separate the Hulu rows from the rest of the df
netflix_tv = streaming_clean[streaming_clean["Service_Name"] == "Netflix"] 
# Group the Genre_1 column and get the total counts for each genre
netflix_tv_genres = netflix_tv.groupby("Genre_1")["Genre_1"].count().reset_index(name = "Count")
# Ascertain the top 5 genres
top_netflix_tv_genres = netflix_tv_genres.sort_values("Count", ascending = False).head(6)

plt.figure(figsize = (8, 6))
sn.barplot(x = top_netflix_tv_genres["Genre_1"], y = top_netflix_tv_genres["Count"])
plt.xlabel("Individual Genres from Genre_1 Column")
plt.ylabel(None)
plt.title("Top 5 Netflix Genres")

plt.show

In [ ]:
# Group the Genre_1 column and get the total counts for each genre
netflix_tv_genres = netflix_tv.groupby("Combined_Genres")["Combined_Genres"].count().reset_index(name = "Count")
# Ascertain the top 5 genres
top_netflix_tv_genres = netflix_tv_genres.sort_values("Count", ascending = False).head(6)

plt.figure(figsize = (8, 6))
sn.barplot(x = top_apple_tv_genres["Combined_Genres"], y = top_apple_tv_genres["Count"])
plt.xlabel("Genres")
plt.ylabel(None)
plt.xticks(rotation = 45)
plt.title("Top 5 Netflix Combined Genres")


plt.show

In [ ]:
# Create variable with value counts of each release year
release_yr_cnts = netflix_tv["Release_Year"].value_counts()

# Convert the index of the Release Years from integers to strings because WordCloud requires strings inputs
release_yr_cnts.index = release_yr_cnts.index.map(str)

# Create the WordCloud specifying 
netflix_tv_wordcloud = WordCloud(width = 800, height = 400, background_color = "black").generate_from_frequencies(release_yr_cnts)

# Setting figure size in inches
plt.figure(figsize = (12, 8))
# Show the created WordCloud with a bilinear interpolation, which is used to estimate the value of pixels used to generate the image
plt.imshow(netflix_tv_wordcloud, interpolation = "bilinear")
# Do not display axes
plt.axis("off")
# Create a title for the WordCloud
plt.title("Netflix Release Years WordCloud")
# Show the plot
plt.show()

In [ ]:
# Separate the Hulu rows from the rest of the df
prime_tv = streaming_clean[streaming_clean["Service_Name"] == "Prime"] 
# Group the Genre_1 column and get the total counts for each genre
prime_tv_genres = prime_tv.groupby("Genre_1")["Genre_1"].count().reset_index(name = "Count")
# Ascertain the top 5 genres
top_prime_tv_genres = prime_tv_genres.sort_values("Count", ascending = False).head(6)

plt.figure(figsize = (8, 6))
sn.barplot(x = top_prime_tv_genres["Genre_1"], y = top_prime_tv_genres["Count"])
plt.xlabel("Individual Genres from Genre_1 Column")
plt.ylabel(None)
plt.title("Top 5 Prime Genres")

plt.show

In [ ]:
# Group the Genre_1 column and get the total counts for each genre
prime_tv_genres = prime_tv.groupby("Combined_Genres")["Combined_Genres"].count().reset_index(name = "Count")
# Ascertain the top 5 genres
top_prime_tv_genres = prime_tv_genres.sort_values("Count", ascending = False).head(6)

plt.figure(figsize = (8, 6))
sn.barplot(x = top_prime_tv_genres["Combined_Genres"], y = top_prime_tv_genres["Count"])
plt.xlabel("Genres")
plt.ylabel(None)
plt.xticks(rotation = 45)
plt.title("Top 5 Prime Combined Genres")

plt.show()

In [ ]:
# Create variable with value counts of each release year
release_yr_cnts = prime_tv["Release_Year"].value_counts()

# Convert the index of the Release Years from integers to strings because WordCloud requires strings inputs
release_yr_cnts.index = release_yr_cnts.index.map(str)

# Create the WordCloud specifying 
prime_tv_wordcloud = WordCloud(width = 800, height = 400, background_color = "black").generate_from_frequencies(release_yr_cnts)

# Setting figure size in inches
plt.figure(figsize = (12, 8))
# Show the created WordCloud with a bilinear interpolation, which is used to estimate the value of pixels used to generate the image
plt.imshow(prime_tv_wordcloud, interpolation = "bilinear")
# Do not display axes
plt.axis("off")
# Create a title for the WordCloud
plt.title("Prime Release Years WordCloud")
# Show the plot
plt.show()

In [ ]:
# streaming_clean[streaming_clean["Title"] == "Tracker"]
streaming_clean[streaming_clean["Title"].isin(["Tracker", "Elsbeth"])]


In [ ]:
streaming_clean.dtypes